## Material optimization

In this notebook we calibrate the spectrally varying absorption and scattering coefficient of a wall in order to match the simulated ETC to a simulated target ETC.

## Overview

Misuka can be used to solve inverse problems using a technique known as *differentiable rendering*. It interprets the rendering algorithm as a function $f(\mathbf{\pi})$ that converts an input $\mathbf{\pi}$ (the scene description) into an output $H(t)$, the Energy Time Curve (ETC). This function $f$ is then mathematically differentiated to obtain $\frac{\partial H}{\partial \mathbf{\pi}}$, providing a first-order approximation of how a desired change in the output $H(t)$ (the rendering) can be achieved by changing the inputs $\mathbf{\pi}$ (the scene description). Together with a differentiable *objective function* $\mathcal{L}(H)$ that quantifies the suitability of tentative scene parameters, a gradient-based optimization algorithm such as stochastic gradient descent or Adam can then be used to find a sequence of scene parameters $\mathbf{\pi}_0$, $\mathbf{\pi}_1$, $\mathbf{\pi}_2$, etc., that successively improve the objective function.

<div align="center"><img src="../../resources/data_acoustic/docs/images/autodiff/autodiff_figure.jpg" width="60%"/></div>

In this tutorial, we will build a simple example application that showcases differentiation and optimization through an acoustic simulation:

1. We will first render a reference ETC of a simple scene with just one reflecting surface.
2. Then, we will change the absorption and scattering coefficient of the surface.
3. Finally, we will try to recover the original absorption and scattering coefficients of the surface using differentiation along with the reference ETC generated in step 1.

<div class="admonition important alert alert-block alert-success">

🚀 **You will learn how to:**
    
<ul>
  <li>Build a simple scene with python only</li>
  <li>Update a surface's absorption and scattering coefficient with a function</li>
  <li>Build an optimization loop using the <code>Optimizer</code> classes</li>
  <li>Perform a gradient-based optimization using automatic differentiation</li>
</ul>
    
</div>

    
[1]: https://drjit.readthedocs.io/en/master/

## Setup

In order to use the automatic differentiation, we need to enable a variant that supports it. Those are the ones containing `_ad` after the backend description. E.g. `cuda_ad_acoustic`, `llvm_ad_acoustic`, …

<div class="admonition important alert alert-block alert-info">
    
If you receive an error mentioning that the requested variant is not supported, add the desired variant to your `build/mitsuba.conf` file and recompile the project ([documentation][1]).

[1]: https://mitsuba.readthedocs.io/en/latest/src/developer_guide/compiling.html#configuring-mitsuba-conf

</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import drjit as dr
import mitsuba as mi
mi.set_variant('cuda_ad_acoustic', 'llvm_ad_acoustic')
from mitsuba import ScalarTransform4f as tf # needed to move and scale geometry

# Fixed rng seeds for reproducibility
seed   = 0

## Scene Construction

First, we set up the scene geometry using Mitsuba's basic rectangle shape. We place the reflector in the 

In [ ]:
wall_pos    = np.array([ 0, 0, 0])
mic_pos     = np.array([ 5, 0, 5])
speaker_pos = np.array([-5, 0, 5])

scene_dict = {
    'type': 'scene',

    'speaker': {
        'type': 'sphere',
        'center': speaker_pos,
        'radius': 2,
        'emitter': {
            'type': 'area',
            'radiance': {
                'type': 'uniform',
                'value': 100.,
            }
        },
    },

    'reflector': {
        'type': 'rectangle',
        'to_world': tf().scale(20).translate(wall_pos), # Transformations are applied from right to left
    },
}


Here, we will use Mitsuba's BlendBSDF to model a specular + diffuse reflection BSDF that is commonly used in acoustics. A few details require special attention:

- In acoustics, the absorption coefficient $\alpha$ expresses the fraction of energy absorbed by a material. Mitsuba's BSDFs use the reflectance $R$, which expresses the of reflected energy. The two are related by $R = 1 - \alpha$.
- The BlendBSDF combines the specular and diffuse contributions using its weight parameter. Since it does not support spectrally varying weights, we use a constant value here. An acoustic BSDF with support for spectrally varying scattering coefficients will be added to misuka in the near future.
- In order to correctly propagate gradients through the BSDF, we use the differentiable `roughconductor` BSDF (left image). However, we use a very low roughness (`alpha`) value so we effectively converge to a specular reflection (right image).

<div align="center">
<img src="../../resources/data_acoustic/docs/images/bsdf/acoustic_bsdf_rough.jpg" width="30%"/>
<img src="../../resources/data_acoustic/docs/images/bsdf/acoustic_bsdf.jpg" width="30%"/>
</div>

Spectrally varying parameters (like the reflectance) can be imported as a list of frequency-value pairs.

In [ ]:
absorption_target  = [(125, 0.1), (250, 0.4), (500, 0.7), (1000, 0.9)]
reflectance_target = [(i, 1 - j) for i, j in absorption_target]
frequencies        = [i[0] for i in absorption_target] # we will use this later for plot legends
scattering_target  = 0.1

# Add the BSDF to the scene dictionary
scene_dict['reflector']['bsdf'] = {
    'type': 'blendbsdf',
    'weight': scattering_target,
    'bsdf_0': {
        'type': 'roughconductor',
        'alpha': 0.01,
        'specular_reflectance': {
            'type': 'spectrum',
            'value': reflectance_target,
        },
    },
    'bsdf_1': {
        'type': 'diffuse',
        'reflectance': {
            'type': 'spectrum',
            'value': reflectance_target,
        }
    },
}

Now we can load the scene:

In [ ]:
scene = mi.load_dict(scene_dict)

### Sensor and Integrator

Now that the scene geometry is complete, we need to define the sensor and integrator.

In [ ]:
max_time = 0.1
sampling_rate = 5000
t_bins = int(max_time * sampling_rate)

microphone = mi.load_dict({
    'type': 'microphone',
    'to_world': tf().translate(mic_pos),
    'film': {
        'type': 'tape',
        'frequencies':  '125, 250, 500, 1000',  # Frequencies that we want to render
        'time_bins': t_bins,                    # Number of time bins to use
        'rfilter':   {'type': 'box'},           # no smoothing in the time domain
    },
})

integrator_acoustic = mi.load_dict({
    'type': 'acoustic_prb',          # Acoustic Path Replay Backpropagation
    'max_depth': 2,                  # Stopping criterion for the max reflection depth, 1 = direct sound, 2 = 1st order
    'max_time': max_time,            # Stopping criterion for the maximum propagation time
    'track_time_derivatives': False, # In a static scene we do not need to track time derivatives
    'skip_direct': False,            # We can skip the direct sound if we want to
})


### Reference Energy Time Curve

We render a reference ETC of the original scene that will later be used in the objective function for the optimization. 
Ideally, this reference ETC should contain very little noise as it will hinder the optimization process otherwise. 
For best results, we should to render it with with as many samples as possible.

In [ ]:
spp_ref = 2**26
etc_ref = mi.render(scene,sensor=microphone, integrator=integrator_acoustic, spp=spp_ref) / spp_ref

In [ ]:
def plot_etc(etc: mi.TensorXf, sampling_rate: int, max_time: float,
             dB: bool = True, normalize: bool = False,
             bottom: float = -100, ax=None, **kwargs):
    """Plot Energy Time Curve"""
    times = np.arange(max_time, step=1/sampling_rate)
    etc = etc.numpy().squeeze()

    if ax is None:
        ax = plt.gca()
    ax.set_xlabel('Time in s')
    ax.set_ylabel('Amplitude')

    if normalize:
        etc  /= np.max(etc)

    if dB:
        with np.errstate(divide='ignore'):
            etc = 20 * np.nan_to_num(np.log10(np.abs(etc)), neginf=-300)
        ax.set_ylabel('Amplitude in dB')

    ax.plot(times, etc, **kwargs)
    ax.set_xlim(0, max_time)
    ax.set_ylim(bottom=bottom, top=np.max(etc) + 10)

In [ ]:
plot_etc(etc_ref, sampling_rate, max_time, dB=True, normalize=False, bottom=-100, label=frequencies)
plt.legend(title = 'Frequency in Hz')
plt.show()

### Initial State

In order to optimize the material properties, we set up an Adam optimizer and define the initial values. The number of absorption values must match the number of frequency nodes we defined in the spectral values above.

In [ ]:
opt = mi.ad.Adam(lr=0.005)
opt['absorption'] = mi.Float([0.5, 0.5, 0.5, 0.5])
opt['scattering'] = mi.Float(0.8)

Using the `traverse` mechanism, we can pick the parameters that we will be optimizing and change its value away from the correct value. The goal of the optimization process will be to recover the original value of this parameter using gradient descent.

Because we need to update both the scattering coeffiecient (`weight`) and the reflectance of the specular and diffuse part of the BSDF (`bsdf_0.specular_reflectance`, `bsdf_1.reflectance`), it is convenient to write a small function that updates all values at once

The `update_material_properties()` function ensures that:
1. All coefficients remain within physical bounds $[0, 1]$
2. The reflectance values are updated as `1 - absorption` for both specular and diffuse components
3. The scattering weight controls the blend between specular and diffuse reflection
4. Changes are applied to the scene parameters through `params.update()`

In [ ]:
params = mi.traverse(scene)

def update_material_properties(params: mi.SceneParameters, opt: mi.ad.Optimizer):
    """Write absorption and scattering coefficient to the scene parameters"""

    # clip values to [0, 1]
    opt['absorption'] = dr.clip(opt['absorption'], 0.0, 1.0)
    opt['scattering'] = dr.clip(opt['scattering'], 0.0, 1.0)

    # update the parameter dictionary
    params['reflector.bsdf.bsdf_0.specular_reflectance.values'] = 1-opt['absorption']
    params['reflector.bsdf.bsdf_1.reflectance.values']          = 1-opt['absorption']
    params['reflector.bsdf.weight.value']                       = opt['scattering']

    params.update(opt)

After updating we can render the scene again and see that the initial condition has been applied.

In [ ]:
update_material_properties(params, opt)
etc_initial = mi.render(scene, integrator=integrator_acoustic, sensor=microphone, spp=spp_ref) / spp_ref
plot_etc(etc_initial, sampling_rate, max_time, dB=True, normalize=False, bottom=-100, label=frequencies)
plt.legend(title = 'Frequency in Hz')
plt.show()

## Optimization

At every iteration of the gradient descent, we will compute the derivatives of the scene parameters with respect to the objective function. In this simple experiment, we use the total squared error between the current ETC and the reference created above.

In [ ]:
def mse(etc):
    return dr.mean(dr.square(etc - etc_ref))

In the following cell we define the hyper parameters controlling our optimization loop, such as the the number of samples and number of iterations:

In [ ]:
spp_opt = 2**18
iteration_count = 400

In [ ]:
# IGNORE THIS: When running under pytest, adjust parameters to reduce computation time
import os
if 'PYTEST_CURRENT_TEST' in os.environ:
    iteration_count = 2

### Optimization Loop
It is now time to actually perform the gradient-descent loop.

We first set up empty arrays to store the optimization progress and a self-updating plot. The dashed lines show the target values.

In [ ]:
absorption_values         = np.empty((iteration_count, len(absorption_target)))
scattering_values, losses = np.empty(iteration_count), np.empty(iteration_count)

%matplotlib widget
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(10, 3), width_ratios=[1,1,1,1.5], layout='constrained')
ax1.set_xlim(-iteration_count * 0.02, iteration_count * 1.02)
ax2.set_xlim(-iteration_count * 0.02, iteration_count * 1.02)
ax3.set_xlim(-iteration_count * 0.02, iteration_count * 1.02)

for f in range(len(absorption_target)):
    ax1.axhline(absorption_target[f][1], linestyle='dotted', color=f'C{f}')
ax2.axhline(scattering_target, linestyle='dotted')

ax1.set_ylim(-0.1, 1.1)
ax2.set_ylim(-0.1, 1.1)
ax3.set_ylim(-0.1, 1.1)

ax1.set_xlabel('Iteration')
ax2.set_xlabel('Iteration')
ax3.set_xlabel('Iteration')
ax4.set_xlabel('Time in s')
ax1.set_ylabel(rf'Absorption coefficient')
ax2.set_ylabel(rf'Scattering coefficient')
ax3.set_ylabel('Loss')
ax4.set_ylabel('ETC in dB')
plt.show()

It is now time to run the optimization loop.

In order to keep the (noisy) simulations statistically independent and to prevent overfitting we use a new rng seed for each iteration. This produces different noise patterns in each iteration.
At some point the gradients become smaller than the noise, indicating that our optimization has converged. If we want to improve the quality of the optimization we need to increase the number of samples per iteration `spp_opt`.

In [ ]:
for i in range(iteration_count):
    # Perform a (noisy) differentiable rendering of the scene. Use a different rng seed for each iteration
    etc = mi.render(scene, params, integrator=integrator_acoustic, sensor=microphone,
                    spp=spp_opt, seed=seed+i) / spp_opt

    # Evaluate the objective function
    loss = mse(etc)

    # Store the current absorption and scattering values
    absorption_values[i] = opt['absorption']
    scattering_values[i] = opt['scattering'][0]
    losses[i] = loss.numpy()

    # Backpropagate the loss through the rendering process
    dr.backward(loss)

    # Take a gradient descent step
    opt.step()

    # Update the scene
    update_material_properties(params, opt)

    # Update the plot
    ax1.clear()
    ax2.clear()
    ax3.clear()
    ax4.clear()

    ax1.plot(absorption_values[:i])
    for f in range(len(absorption_target)):
        ax1.axhline(absorption_target[f][1], linestyle='dotted', color=f'C{f}')
    ax2.plot(scattering_values[:i])
    ax2.axhline(scattering_target, linestyle='dotted')
    ax3.semilogy(losses[:i])
    plot_etc(etc_ref, sampling_rate, max_time, dB=True, normalize=False, bottom=-100,
             ax=ax4, linestyle='dotted', label='_nolabel')
    ax4.set_prop_cycle(None) # reset plot colors
    plot_etc(etc, sampling_rate, max_time, dB=True, normalize=False, bottom=-100,
             ax=ax4, label=frequencies)

    ax1.set_xlim(-iteration_count * 0.02, iteration_count * 1.02)
    ax2.set_xlim(-iteration_count * 0.02, iteration_count * 1.02)
    ax3.set_xlim(-iteration_count * 0.02, iteration_count * 1.02)
    ax4.set_xlim(0.02, 0.08)

    ax1.set_ylim(-0.1, 1.1)
    ax2.set_ylim(-0.1, 1.1)
    ax4.set_ylim(-100, -20)

    ax1.set_xlabel('Iteration')
    ax2.set_xlabel('Iteration')
    ax3.set_xlabel('Iteration')
    ax1.set_ylabel(rf'Absorption coefficient')
    ax2.set_ylabel(rf'Scattering coefficient')
    ax3.set_ylabel('Loss')
    ax4.legend(title='Frequency in Hz')

    fig.canvas.draw()